A rotatividade de clientes é um problema crucial no setor de telecomunicações, definido como a perda de clientes para outra operadora. Prever a rotatividade antecipadamente, como "este cliente vai nos deixar nos próximos X meses?", permite aplicar políticas de marketing para reter clientes e aumentar a base. Mesmo uma redução de 1% na taxa de rotatividade pode resultar em um aumento significativo nos lucros.

A análise de churn envolve estudar "um cliente", "um produto ou serviço" e "a probabilidade de abandono". O objetivo é identificar clientes propensos a sair e tomar ações preventivas antes que eles partam. 

Os dados de rotatividade que utilizo são de uma empresa fictícia que forneceu serviços de telefone e Internet para 7.043 clientes na Califórnia no terceiro trimestre. Com esses dados, desenvolvo um modelo de aprendizado de máquina para prever quais clientes deixarão a empresa, realizando análise de dados e engenharia de recursos.

O conjunto de dados Telco Customer Churn da Kaggle foi utilizado, contendo 21 colunas e 7.043 linhas com informações como customerID, sexo, serviço telefônico e de Internet. Analiso essas colunas para identificar variáveis independentes (X) e dependentes.

Esse projeto me proporciona experiência prática em ciência de dados, incluindo a manipulação de grandes volumes de dados, aplicação de técnicas de aprendizado de máquina, e implementação de estratégias de retenção de clientes, preparando-me para atuar em diversos setores e segmentos de negócios.


### 0.0 - Imports

In [17]:
# mmanipulação
import pandas as pd
import inflection
import gc

#Normaçizaçõe e divisões
from sklearn.model_selection import train_test_split

# matematica e estatistica
import numpy as np
from scipy import stats

# visualização
import matplotlib.pyplot as plt
import seaborn as sns

gc.collect()

483

### 0.1 - Help Functions

In [2]:
def show_descriptive_statistical(df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

### 0.2 - Path Definition

In [3]:
HOME_PATH = 'C:/Users/valmi/projeto_churn/'
RAW_DATA_PATH = '/data/raw/'
INTERIM_DATA_PATH = '/data/interim/'
FIGURE_PATH = '/reports/figures/'

### 0.3- Load Data

In [12]:
base_churn_raw = pd.read_csv(HOME_PATH+RAW_DATA_PATH+'WA_Fn-UseC_-Telco-Customer-Churn.csv')
base_churn_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### 0.4 - Divisão Train e Test

#### 0.4.1 - Transformação do Tipo da variavel target

In [14]:
base_churn_raw['Churn'] = base_churn_raw['Churn'].apply(lambda x:1 if x== 'Yes' else 0)

#### 0.4.2 - Divisão 

In [18]:
x = base_churn_raw.drop("Churn", axis=1)
y = base_churn_raw["Churn"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

base_churn_train = pd.concat([X_train,y_train], axis=1)
base_churn_teste = pd.concat([X_test,y_test], axis=1)

In [19]:
base_churn_train.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3738,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,0
3151,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,0
4860,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,0
3867,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.7,0
3810,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,0


### 1.0 - Data Descripition

In [21]:
base_churn_raw1 = base_churn_train.copy()
base_churn_raw1.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_churn_raw1.csv')
base_churn_raw1 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_churn_raw1.csv')


| Atributo       | Significado                                                                               |
| -------------- | ----------------------------------------------------------------------------------------- |
|customerID      | valor único que identifica o cliente														 |	
|genero          |se o cliente é homem ou mulher                                                             |
|SeniorCitizen   |se o cliente é idoso ou não (1, 0)                                                         |
|Parceiro        |se o cliente tem parceiro ou não (Sim, Não) 												 |
|Dependentes     |se o cliente possui dependentes ou não (Sim, Não)                                          |
|fidelidade      |número de meses que o cliente permaneceu na empresa                                        |
|PhoneService    |se o cliente possui serviço telefônico ou não (Sim, Não)                                   |
|MultipleLines   |se o cliente possui múltiplas linhas ou não (Sim, Não, Não serviço telefônico)             |
|InternetService |provedor de serviços de internet do cliente (DSL, Fibra óptica, Não)                       |
|OnlineSecurity  |se o cliente possui segurança online ou não (Sim, Não, Sem serviço de internet)            |
|OnlineBackup    |se o cliente possui backup online ou não (Sim, Não, Sem serviço de internet)               | 
|DeviceProtection|se o cliente possui proteção de dispositivo ou não (Sim, Não, Sem serviço de internet)     |
|TechSupport     |se o cliente tem suporte técnico ou não (Sim, Não, Sem serviço de internet)                |
|StreamingTV     |se o cliente possui streaming de TV ou não (Sim, Não, Não serviço de internet)             |
|StreamingMovies |se o cliente possui streaming de filmes ou não (Sim, Não, Sem serviço de internet)         |
|Contrato        |tipo de contrato de acordo com a duração (Mês a mês, Um ano, Dois anos)                    |
|PaperlessBilling|faturas emitidas em formato sem papel (Sim, Não)                                           |
|PaymentMethod   |forma de pagamento utilizada pelo cliente                                                  |
|MonthlyCharges  |valor cobrado pelo serviço mensalmente                                                     |  
|TotalCharges    |cobranças cumulativas de serviço durante o período de assinatura (manutenção)              |



### 1.2 - Rename Columns

In [22]:
base_churn_raw1.rename(columns={'gender':'genero', 'SeniorCitizen':'CliIdoso','Partner':'Parceiro',
								'tenure':'Fidelidade', 'DeviceProtection':'ProtecaoDispositivo', 'StreamingMovies':'StreamFilmes', 
								'PaperlessBilling':'CobrancaPapel','PaymentMethod':'FormaPagCliente',
								'MonthlyCharges':'VlCobrancaMensal','TotalCharges':'CobrancaCumulativa'})

,Unnamed: 0,customerID,genero,CliIdoso,Parceiro,Dependents,Fidelidade,PhoneService,MultipleLines,InternetService,...,ProtecaoDispositivo,TechSupport,StreamingTV,StreamFilmes,Contract,CobrancaPapel,FormaPagCliente,VlCobrancaMensal,CobrancaCumulativa,Churn
0,3738,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,0
1,3151,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,0
2,4860,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,0
3,3867,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.7,0
4,3810,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6303,6308-CQRBU,Female,0,Yes,No,71,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Two year,No,Electronic check,109.25,7707.7,0
5630,6227,2842-JTCCU,Male,0,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,No,Bank transfer (automatic),46.05,80.35,1
5631,4673,6402-ZFPPI,Female,1,No,No,25,Yes,Yes,Fiber optic,...,No,No,Yes,Yes,Month-to-month,Yes,Mailed check,102.80,2660.2,1
5632,2710,3594-BDSOA,Female,0,Yes,No,24,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),20.40,482.8,0


### 1.3 - Data Dimensions

In [23]:
num_linhas, num_colunas = base_churn_raw1.shape

print('Numero de linhas', num_linhas)
print('Numero de colunas', num_colunas)

Numero de linhas 5634
Numero de colunas 22


### 1.4 - Data Types

In [24]:
base_churn_raw1.dtypes

Unnamed: 0            int64
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [25]:
del base_churn_raw1['Unnamed: 0']

In [26]:
base_churn_raw1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,0
1,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,0
2,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,0
3,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.7,0
4,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,0


In [28]:
categorical_columns = base_churn_raw1.select_dtypes(include=[object]).columns
print("\nColunas categóricas antes da conversão:\n", categorical_columns)


Colunas categóricas antes da conversão:
 Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges'],
      dtype='object')


In [36]:
# Identificando colunas categóricas
categorical_columns = base_churn_raw1.select_dtypes(include=['object']).columns

# Visualizando os valores únicos de cada variável categórica
for column in categorical_columns:
    unique_values = base_churn_raw1[column].unique()
    print(f"Valores únicos em '{column}': {unique_values}\n")

Valores únicos em 'customerID': ['4950-BDEUX' '7993-NQLJE' '7321-ZNSLA' ... '6402-ZFPPI' '3594-BDSOA'
 '6490-FGZAT']

Valores únicos em 'gender': ['Male' 'Female']

Valores únicos em 'Partner': ['No' 'Yes']

Valores únicos em 'Dependents': ['No' 'Yes']

Valores únicos em 'PhoneService': ['No' 'Yes']

Valores únicos em 'MultipleLines': ['No phone service' 'No' 'Yes']

Valores únicos em 'InternetService': ['DSL' 'Fiber optic' 'No']

Valores únicos em 'OnlineSecurity': ['No' 'Yes' 'No internet service']

Valores únicos em 'OnlineBackup': ['No' 'Yes' 'No internet service']

Valores únicos em 'DeviceProtection': ['Yes' 'No' 'No internet service']

Valores únicos em 'TechSupport': ['No' 'Yes' 'No internet service']

Valores únicos em 'StreamingTV': ['Yes' 'No' 'No internet service']

Valores únicos em 'StreamingMovies': ['Yes' 'No' 'No internet service']

Valores únicos em 'Contract': ['Month-to-month' 'Two year' 'One year']

Valores únicos em 'PaperlessBilling': ['No' 'Yes']

Valores únicos

``CONSIDERAÇÕES``

 - Colunas com Respostas Sim/Não: 
 
 Muitas colunas categóricas podem ter respostas binárias (ex.: Yes/No), que podem ser transformadas em variáveis numéricas (0/1) posteriormente.

 - Colunas com Múltiplas Categorias: 
 
 Colunas com mais de duas categorias (como InternetService) podem exigir uma abordagem de codificação, como One-Hot Encoding.

In [40]:
base_churn_raw1['Partner'] = base_churn_raw1['Partner'].apply(lambda x:1 if x== 'Yes' else 0)
base_churn_raw1['Dependents'] = base_churn_raw1['Dependents'].apply(lambda x:1 if x== 'Yes' else 0)
base_churn_raw1['PhoneService'] = base_churn_raw1['PhoneService'].apply(lambda x:1 if x== 'Yes' else 0)
base_churn_raw1['PaperlessBilling'] = base_churn_raw1['PaperlessBilling'].apply(lambda x:1 if x== 'Yes' else 0)



In [41]:
# Identificando colunas categóricas
categorical_columns = base_churn_raw1.select_dtypes(include=['object']).columns

# Visualizando os valores únicos de cada variável categórica
for column in categorical_columns:
    unique_values = base_churn_raw1[column].unique()
    print(f"Valores únicos em '{column}': {unique_values}\n")

Valores únicos em 'customerID': ['4950-BDEUX' '7993-NQLJE' '7321-ZNSLA' ... '6402-ZFPPI' '3594-BDSOA'
 '6490-FGZAT']

Valores únicos em 'gender': ['Male' 'Female']

Valores únicos em 'MultipleLines': ['No phone service' 'No' 'Yes']

Valores únicos em 'InternetService': ['DSL' 'Fiber optic' 'No']

Valores únicos em 'OnlineSecurity': ['No' 'Yes' 'No internet service']

Valores únicos em 'OnlineBackup': ['No' 'Yes' 'No internet service']

Valores únicos em 'DeviceProtection': ['Yes' 'No' 'No internet service']

Valores únicos em 'TechSupport': ['No' 'Yes' 'No internet service']

Valores únicos em 'StreamingTV': ['Yes' 'No' 'No internet service']

Valores únicos em 'StreamingMovies': ['Yes' 'No' 'No internet service']

Valores únicos em 'Contract': ['Month-to-month' 'Two year' 'One year']

Valores únicos em 'PaymentMethod': ['Electronic check' 'Mailed check' 'Credit card (automatic)'
 'Bank transfer (automatic)']

Valores únicos em 'TotalCharges': ['1701.65' '1151.55' '590.35' ... '80.35' 